# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS data_modeling_apache_cassandra
    WITH REPLICATION = { 'class': 'SimpleStrategy', 'replication_factor': 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('data_modeling_apache_cassandra')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
For this query the we need the artist name, the song title and the song length. So these columns need to be in the new table, but none of these are used to filter the table, so they are'nt part of the primary key. The columns contained in primary key are session id and item in session. These columns garantee for as the possibilite of accomplish the query above without a problem of duplicate rows


### First step: Here we'll use the CREATE statement to create a new TABLE

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_library "
query = query + "(artist text, song_title text, length float, session_id int, item_in_session int, PRIMARY KEY (session_id, item_in_session))"     

try:
    session.execute(query)
except Exception as e:
    print(e)

### Second step: Open the CSV file and use each row(except the first which contains the header) to create a INSERT and add this value into table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_library (artist, song_title, length, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        #Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

### Third step: Do the SELECT specified above to verify that the data have been inserted into table and there was no problem

In [10]:
query = 'SELECT artist, song_title, length FROM session_library WHERE session_id = 338 AND item_in_session = 4'

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print('Artist: {}. \nSong title: {}. \nLength: {}.'.format(row.artist, row.song_title, row.length))

Artist: Faithless. 
Song title: Music Matters (Mark Knight Dub). 
Length: 495.30731201171875.


## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
For this query the we need the artist name, the song title and the user first and last name. These columns need to be in the new table. 
The first two columns in primary key are user id and session id, both are used to filter the data. Futhermore, it's specified that this data will be order by the value of item in session, so we deduce this is part of the primary key too. So we have a composite primary key, created from these three columns in this order.

### First step: Here we'll use the CREATE statement to create a new TABLE

In [8]:
query = "CREATE TABLE IF NOT EXISTS user_library "
query = query + "(artist text, song_title text, item_in_session int, first_name text, last_name text, user_id int, session_id int, PRIMARY KEY(user_id, session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

### Second step: Open the CSV file and use each row(except the first which contains the header) to create a INSERT and add this value into table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_library (artist, song_title, item_in_session, first_name, last_name, user_id, session_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        #Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8])))     

### Third step: Do the SELECT specified above to verify that the data have been inserted into table and there was no problem

In [10]:

query = 'SELECT artist, song_title, item_in_session, first_name, last_name FROM user_library WHERE user_id = 10 AND session_id = 182'

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print('Item in Session: {}. \nArtist: {}. \nSong title: {}. \nUser First Name: {}. \nUser Last Name: {}. \n\n'
          .format(row.item_in_session, row.artist, row.song_title, row.first_name, row.last_name))   

Item in Session: 0. 
Artist: Down To The Bone. 
Song title: Keep On Keepin' On. 
User First Name: Sylvie. 
User Last Name: Cruz. 


Item in Session: 1. 
Artist: Three Drives. 
Song title: Greece 2000. 
User First Name: Sylvie. 
User Last Name: Cruz. 


Item in Session: 2. 
Artist: Sebastien Tellier. 
Song title: Kilometer. 
User First Name: Sylvie. 
User Last Name: Cruz. 


Item in Session: 3. 
Artist: Lonnie Gordon. 
Song title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit). 
User First Name: Sylvie. 
User Last Name: Cruz. 




## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
For the third query the we need the user first and last name. These columns need to be in the new table. 
Other column needed is the song title, used to filter the table in the query above, so this is our partition key. This structure already give us the possibilite of return some results, but in this case there is a possibilite of there are multiples users with a same name, so we garantee that this won't happen adding the user id column as a clustering column in our composite primary key to avoid the chance of duplicate inserts.

### First step: Here we'll use the CREATE statement to create a new TABLE

In [15]:
query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(first_name text, last_name text, song_title text, user_id int, PRIMARY KEY(song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

                    

### Second step: Open the CSV file and use each row(except the first which contains the header) to create a INSERT and add this value into table

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_library (first_name, last_name, song_title, user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        #Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[1], line[4], line[9], int(line[10])))   

### Third step: Do the SELECT specified above to verify that the data have been inserted into table and there was no problem

In [18]:
# SELECT statement to verify the data was entered into the table
query = "SELECT first_name, last_name FROM song_library WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print('User First Name: {}. \nUser Last Name: {}. \n\n'.format(row.first_name, row.last_name))

User First Name: Jacqueline. 
User Last Name: Lynch. 


User First Name: Tegan. 
User Last Name: Levine. 


User First Name: Sara. 
User Last Name: Johnson. 




### Drop the tables before closing out the sessions

In [19]:
query = "DROP TABLE session_library"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE user_library"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE song_library"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()